In [1]:
import numpy as np
from sklearn.semi_supervised import LabelSpreading
from sklearn.model_selection import train_test_split
from sklearn.semi_supervised import LabelPropagation
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import os
import pandas as pd
from sklearn.metrics import average_precision_score

In [2]:
%pwd

'/Users/glrz'

In [3]:
#load dataset

In [36]:
#import file
os.chdir("/Users/glrz/Desktop/Thesis/src/base")
file_train = "train_u_with_days.csv"
file_test = "test_with_days.csv"
train = pd.read_csv(file_train, sep = ',')
test = pd.read_csv(file_test, sep = ',')

val_size = len(test)

train, val = train_test_split(train, test_size = val_size, shuffle = True, random_state=42)


In [37]:
#undersample train
limit = 70000
idx_normal = train[train['outcome'] == 0].index
permutation = np.random.permutation(limit)
train_normal_samples = train.loc[idx_normal[permutation]]
train_b = pd.concat([train_normal_samples,train[train['outcome'] == 1]])

In [38]:
#oversample test
#idx_outlier = train[train['outcome'] == 1].index
#train_outlier = train[train['outcome'] == 1]
#train_b = pd.concat([train_outlier,train_outlier,train_outlier,train)

In [39]:
y_train = train_b['outcome']
x_train = train_b.drop(['outcome'], axis=1)
y_val = val['outcome']
x_val = val.drop(['outcome'], axis=1)
y_test = test['outcome']
x_test = test.drop(['outcome'], axis=1)
val_size = len(y_test)

semi_labels = y_train.copy()
x_train.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
idx_normal = y_train[y_train == 0].index
idx_outlier = y_train[y_train == 1].index

In [92]:
#creating 25 labeled, 75 unlabeled train set

ratio_known_normal = 0.50
ratio_unknown_normal = 0.50
ratio_known_outlier = 0.50
ratio_unknown_outlier = 0.50

n_complete_dataset_normal = len(idx_normal)
n_complete_dataset_outlier = len(idx_outlier)
n_known_normal = int(n_complete_dataset_normal * ratio_known_normal)
n_unlabeled_normal = min(n_complete_dataset_normal - n_known_normal,
                             round(ratio_unknown_normal * n_complete_dataset_normal))
n_known_outlier = round(ratio_known_outlier * n_complete_dataset_outlier)
n_unlabeled_outlier = min(n_complete_dataset_outlier - n_known_outlier,
                              round(ratio_unknown_outlier * n_complete_dataset_outlier))
# using permutation to randomly assign between groups
perm_normal = np.random.permutation(n_complete_dataset_normal)
perm_known_outlier = np.random.permutation(n_complete_dataset_outlier)
# define the unlabeled indices to return
idx_known_outlier = idx_outlier[
        perm_known_outlier[: n_known_outlier]].tolist()
idx_known_normal = idx_normal[perm_normal[: n_known_normal]].tolist()
idx_unlabeled_outlier = idx_outlier[
    perm_known_outlier[n_known_outlier: n_known_outlier + n_unlabeled_outlier]].tolist()
idx_unlabeled_normal = idx_normal[perm_normal[n_known_normal: n_known_normal + n_unlabeled_normal]].tolist()

print('normal random', len(idx_known_normal), 'random outlier', len(idx_known_outlier), 'unlabeled',
          len(idx_unlabeled_outlier), len(idx_unlabeled_normal))

data = {'idx_known_outlier': idx_known_outlier,
        'idx_known_normal': idx_known_normal,
        'idx_unlabeled_outlier': idx_unlabeled_outlier,
        'idx_unlabeled_normal': idx_unlabeled_normal}

index_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))

normal random 35000 random outlier 8338 unlabeled 8338 35000


In [93]:
#save experiments
#index_df.to_csv('index_df_experiment_1.csv')
#df_train.to_csv('train_df_experiment_1.csv')
#df_validation.to_csv('validation_df_experiment_1.csv')

In [94]:
#loading experiments data
#for iteration_num in {1..10}
#iteration_num=1
#do
    #echo "Itration $i"
#index_experiment = pd.read_csv(f'index_df_experiment_{iteration_num}.csv')

#idx_known_outlier = index_experiment['known_outlier'].dropna().astype(int)
#idx_unlabeled_outlier = index_experiment['unlabeled_outlier'].dropna().astype(int)
#idx_known_normal = index_experiment['known_normal'].dropna().astype(int)
#idx_unlabeled_normal = index_experiment['unlabeled_normal'].dropna().astype(int)

In [95]:
experiment_name = "25_per_labeled_75_unlabeled"
#the auc of my model is ~88 for this case

In [96]:
y_train.index[idx_unlabeled_outlier]

Int64Index([75729, 82502, 70549, 84145, 76542, 74245, 81421, 79503, 73058,
            75888,
            ...
            81190, 81102, 84881, 86398, 71594, 86221, 82152, 78804, 78899,
            77926],
           dtype='int64', length=8338)

In [97]:
#label propagation/ spreading methods
semi_labels.loc[semi_labels.index[idx_unlabeled_outlier]] = -1
semi_labels.loc[semi_labels.index[idx_unlabeled_normal]] = -1

In [98]:
#label propagation params
kernel= ['rbf','knn']
gamma= [5,20,50]
n_neighbors=[3,7,14,20]
max_iter=[1000,5000]

#label spreading param
alpha = [0.2,0.5,0.7]

In [99]:
methods = ['propagation','spreading','self_learning_svc']
chosen_method = methods[1]

In [100]:
#with the best chosen params based on validation set:
if chosen_method=='propagation':
    label_prop_model = LabelPropagation(kernel = kernel[1], n_neighbors= n_neighbors[3], max_iter=max_iter[1])
    label_prop_model.fit(x_train, semi_labels)

#print("pred done", pred[:5])


In [101]:
#labelspreading method
if chosen_method=='spreading':
    label_prop_model = LabelSpreading(kernel = kernel[1], n_neighbors= n_neighbors[3], max_iter=max_iter[0], alpha=alpha[0])
    label_prop_model.fit(x_train, semi_labels)

In [102]:
#compute metrics
y_train_score = label_prop_model.predict_proba(x_train)
y_val_score = label_prop_model.predict_proba(x_val)
y_test_score = label_prop_model.predict_proba(x_test)

/Users/glrz/miniforge3/envs/torchenv/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:231: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer
/Users/glrz/miniforge3/envs/torchenv/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:231: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer
/Users/glrz/miniforge3/envs/torchenv/lib/python3.8/site-packages/sklearn/semi_supervised/_label_propagation.py:231: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer


In [103]:
y_train_m = pd.concat([pd.Series(y_train), pd.Series(y_train_score[:,1])], axis=1)
y_train_m = y_train_m.rename(columns={0:'y_pred_score'})
y_train_m = y_train_m.dropna()

In [104]:
y_val_m = pd.concat([pd.Series(y_val), pd.Series(y_val_score[:,1])], axis=1)
y_val_m = y_val_m.rename(columns={0:'y_pred_score'})
y_val_m = y_val_m.dropna()
y_test_m = pd.concat([pd.Series(y_test), pd.Series(y_test_score[:,1])], axis=1)
y_test_m = y_test_m.rename(columns={0:'y_pred_score'})
y_test_m = y_test_m.dropna()

In [105]:
y_test_m['outcome'].value_counts()

0    52759
1     3572
Name: outcome, dtype: int64

In [106]:
print('propagation train auc', metrics.roc_auc_score(np.array(y_train_m['outcome']).astype(int), np.array(y_train_m['y_pred_score'])))
print('propagation val auc', metrics.roc_auc_score(y_val, y_val_score[:,1]))
print('propagation test auc', metrics.roc_auc_score(np.array(y_test_m['outcome']).astype(int), np.array(y_test_m['y_pred_score'])))

propagation train auc 0.8169765851027604


ValueError: Input contains NaN.

In [ ]:
y_train_predict = label_prop_model.predict(x_train)
y_val_predict = label_prop_model.predict(x_val)
y_test_predict = label_prop_model.predict(x_test)

In [ ]:
y_train_p = pd.concat([pd.Series(y_train), pd.Series(y_train_predict)], axis=1)
y_train_p = y_train_p.rename(columns={0:'y_pred_score'})
y_train_p = y_train_p.dropna()
y_val_p = pd.concat([pd.Series(y_val), pd.Series(y_val_predict)], axis=1)
y_val_p = y_val_p.rename(columns={0:'y_pred_score'})
y_val_p = y_val_p.dropna()
y_test_p = pd.concat([pd.Series(y_test), pd.Series(y_test_predict)], axis=1)
y_test_p = y_test_p.rename(columns={0:'y_pred_score'})
y_test_p = y_test_p.dropna()

In [ ]:
print('propagation train f1', metrics.f1_score(np.array(y_train_p['outcome']).astype(int), np.array(y_train_p['y_pred_score'])))
print('propagation val f1', metrics.f1_score(y_val, y_val_predict))
print('propagation test f1', metrics.f1_score(np.array(y_test_p['outcome']).astype(int), np.array(y_test_p['y_pred_score'])))
    

In [ ]:
print('propagation train aps', metrics.average_precision_score(np.array(y_train_p['outcome']).astype(int), np.array(y_train_p['y_pred_score'])))
print('propagation val aps', metrics.average_precision_score(y_val, y_val_predict))
print('propagation test aps', metrics.average_precision_score(y_test, y_test_predict))

In [ ]:
#self learning svc params
kernel = ["rbf","Nystroem"]
gamma = [0.2,0.5,0.7]


In [30]:
#svc self learning
base_classifier = SVC(probability=True)
self_model = SelfTrainingClassifier(base_classifier)
self_model.fit(x_train, semi_labels)

SelfTrainingClassifier(base_estimator=SVC(probability=True))

In [31]:
y_train_score = self_model.predict_proba(x_train)
y_val_score = self_model.predict_proba(x_val)
y_test_score = self_model.predict_proba(x_test)

In [32]:
print('self svc train auc', metrics.roc_auc_score(y_train, y_train_score[:,1]))
print('self svc val auc', metrics.roc_auc_score(y_val, y_val_score[:,1]))
print('self svc test auc', metrics.roc_auc_score(y_test, y_test_score[:,1]))

self svc train auc 0.8688660243120997
self svc val auc 0.8624507108598197
self svc test auc 0.8657954706403773


In [33]:
y_train_predict = self_model.predict(x_train)
y_val_predict = self_model.predict(x_val)
y_test_predict = self_model.predict(x_test)

In [34]:
print('svc train f1', metrics.f1_score(y_train, y_train_predict))
print('svc val f1', metrics.f1_score(y_val, y_val_predict))
print('svc test f1', metrics.f1_score(y_test, y_test_predict))

svc train f1 0.5942577342532829
svc val f1 0.49335644136337375
svc test f1 0.49347295940324204


In [35]:
print('svc train aps', metrics.average_precision_score(y_train, y_train_predict))
print('svc val aps', metrics.average_precision_score(y_val, y_val_predict))
print('svc test aps', metrics.average_precision_score(y_test, y_test_predict))

svc train aps 0.47417444330893055
svc val aps 0.2764635386309112
svc test aps 0.2765333983126842


In [48]:
x_train.count()

days                262941
tshresult           262941
ft3result           262941
ft4result           262941
tt3result           262941
tt4result           262941
age                 262941
hypertension        262941
dyslipidemia        262941
diabetes            262941
IHD                 262941
CHF                 262941
CVA_TIA             262941
CRF                 262941
chroniclung         262941
chronicliver        262941
BMI                 262941
hyperthyroid        262941
hyperdays           262941
hypothyroid         262941
hypodays            262941
autoanti            262941
autoantidays        262941
hyperdrugdays       262941
hyperdrugcnt        262941
hypodrugdays        262941
hypodrugcnt         262941
smokingdays         262941
SES                 262941
hypodrug_flag       262941
hyperdrug_flag      262941
ft3_flag            262941
ft4_flag            262941
tt3_flag            262941
tt4_flag            262941
is_female           262941
is_smoking          262941
i

In [ ]:
# self training
#https://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.SelfTrainingClassifier.html#sklearn.semi_supervised.SelfTrainingClassifier

In [ ]:
iris.target[random_unlabeled_points] = -1
svc = SVC(probability=True, gamma="auto")
self_training_model = SelfTrainingClassifier(svc)
self_training_model.fit(iris.data, iris.target)